# Showcase how to use SVG Path

https://plot.ly/python/shapes/

https://plot.ly/python/reference/#layout-shapes


Hovertext via workaround:
https://community.plot.ly/t/possible-to-add-tooltips-hovertext-to-shapes/10476

In [1]:
import plotly.graph_objects as go
import xml.etree.ElementTree as ET
from pathlib import Path

import svg.path
import os

In [45]:
folder_preprocessed_data = Path(os.getcwd()).parent / 'data' / 'preprocessed'

In [90]:
fig = go.Figure()

# Create scatter trace of text labels
# fig.add_trace(go.Scatter(
#     x=[2, 1, 8, 8],
#     y=[0.25, 9, 2, 6],
#     text=["Filled Triangle",
#           "Filled Polygon",
#           "Quadratic Bezier Curves",
#           "Cubic Bezier Curves"],
#     mode="text",
# ))

# Update axes properties
fig.update_xaxes(
    range=[0, 1000],
    zeroline=False,
)

fig.update_yaxes(
    range=[0, 1000],
    zeroline=False,
)

# Add shapes
fig.update_layout(shapes=[
    # canton1
    go.layout.Shape(
        name='a',
        type="path",
        path="M 600,700L700,400L200,324.78 Z",
        fillcolor="PaleTurquoise",
        line_color="LightSeaGreen",
    ),
    go.layout.Shape(
        type="path",
        path="M 400,800L500,400L200,424.78 Z",
        fillcolor="PaleTurquoise",
        line_color="LightSeaGreen",
    ),
], showlegend=False)

# fig.add_trace(go.Scatter(
#     x=[400,500,200],
#     y=[800,400,424.78],
#     text = "mario",
#     hoveron="fills",
#     fill="toself",
#     mode="none",
# ))

fig.add_trace(
    go.Scatter(
        x=[400, 500, 200],
        y=[800, 400, 424.78],
        text="mario",
        hoverinfo="text",
        hoveron="fills",
        fill="toself",
        mode="none",
    ))

fig.add_trace(
    go.Scatter(
        x=[800, 900, 600],
        y=[800, 400, 424.78],
        text="mario",
        hoverinfo="text",
        hoveron="fills",
        fill="toself",
        mode="none",
        opacity=0,
    ))
fig.show()

# Parse Swiss map with cantons

https://upload.wikimedia.org/wikipedia/commons/f/f8/Suisse_cantons.svg

Important: Convert the file from inkscape-svg to plain svg! (by using e.g. inkscape)

* layer6 : Cantons
* layer4 : Rivers
* layer3 : Lakes
* layer2 : Frontiers

In [3]:
switzerland_xml = Path(
    os.getcwd()).parent / 'data' / 'map' / 'Suisse_cantons_plain.svg'
root = ET.parse(switzerland_xml).getroot()

Parse Path string:
1. remove tabs & newlines
2. transform to absolute paths (from lowercase to uppercase letters) because Plotly - Paths only understand absolute coordinates

https://github.com/regebro/svg.path/blob/master/src/svg/path/path.py

## Transform relative SVG Path to Absolute

In [43]:
def transform_canton_path(path):
    path = path.replace('\t', '')
    path = path.replace('\n', '')
    canton_parsed = svg.path.parse_path(path)
    path_string_absolute = ''
    for subpath in canton_parsed:
        if type(subpath) == svg.path.path.Move:
            path_string_absolute += f'M{subpath.start.real},{subpath.start.imag}'
        elif type(subpath) == svg.path.path.Line:
            path_string_absolute += f'L{subpath.end.real},{subpath.end.imag}'
        elif type(subpath) == svg.path.path.Close:
            path_string_absolute += 'Z'
        elif type(subpath) == svg.path.path.CubicBezier:
            path_string_absolute += 'C'
            path_string_absolute += f' {subpath.control1.real},{subpath.control1.imag}'
            path_string_absolute += f' {subpath.control2.real},{subpath.control2.imag}'
            path_string_absolute += f' {subpath.end.real},{subpath.end.imag}'
        else:
            print(type(subpath))
    return path_string_absolute

In [60]:
cantons_border_path = {}
for child in root.findall('{http://www.w3.org/2000/svg}g'):
    if child.attrib['id'] == 'layer6':
        for canton in child:
            cantons_border_path[canton.attrib['id']] = transform_canton_path(
                canton.attrib['d'])

In [46]:
with open(folder_preprocessed_data / 'canton_borders.json', 'w') as file:
    file.write(json.dumps(cantons_border_path))

## Transform to X/Y coordinates for border outlines (needed for hover)

In [55]:
def transform_canton_xy(path):
    path = path.replace('\t', '')
    path = path.replace('\n', '')
    canton_parsed = svg.path.parse_path(path)
    xy_borders = []
    xy_border = None
    for subpath in canton_parsed:
        if type(subpath) == svg.path.path.Move:
            # start new xy outline
            xy_border = ([subpath.start.real], [subpath.start.imag])
            xy_borders.append(xy_border)
        elif type(subpath) == svg.path.path.Line or type(
                subpath) == svg.path.path.CubicBezier:
            xy_border[0].append(subpath.end.real)
            xy_border[1].append(subpath.end.imag)
        elif type(subpath) == svg.path.path.Close:
            pass
        else:
            print(type(subpath))
    return xy_borders

In [57]:
cantons_border_xy = {}
for child in root.findall('{http://www.w3.org/2000/svg}g'):
    if child.attrib['id'] == 'layer6':
        for canton in child:
            cantons_border_xy[canton.attrib['id']] = transform_canton_xy(
                canton.attrib['d'])

In [94]:
with open(folder_preprocessed_data / 'canton_borders_xy.json', 'w') as file:
    file.write(json.dumps(cantons_border_xy))

# Show as Plotly Graph

In [93]:
fig = go.Figure(layout={'plot_bgcolor': 'rgba(0,0,0,0)'})

# Update axes properties
fig.update_xaxes(
    range=[0, 1100],
    zeroline=False,
    showgrid=False,
    showticklabels=False,
)

# No idea why, but I have to revert y-axis
fig.update_yaxes(
    range=[700, 0],
    zeroline=False,
    showgrid=False,
    showticklabels=False,
)

canton_shapes = []
for c in cantons_border_path.keys():
    canton_shapes.append(
        go.layout.Shape(type='path',
                        path=cantons_border[c],
                        fillcolor="rgb(15,23,125)",
                        line_color="LightSeaGreen"))

# Add shapes
# fig.update_layout(shapes=canton_shapes, width=990, height=630, showlegend=False)
fig.update_layout(width=990, height=630, showlegend=False)

# Add traces for mouseover
for c in cantons_border_xy.items():
    for subarea in c[1]:
        fig.add_trace(
            go.Scatter(
                x=subarea[0],
                y=subarea[1],
                text=c[0],
                hoverinfo="text",
                hoveron="fills",
                fill="toself",
                mode="none",
                opacity=1,
            ))

fig.show()

In [73]:
for c in cantons_border_xy.items():
    print(c[0])

OW
NW
ZG
LU
AG
BL
VS
BE
SO
JU
VD
FR
BS
GE
NE
GR
SG
SZ
AI
AR
TG
ZH
SH
GL
UR
TI


In [9]:
import pandas as pd

In [10]:
df_ausgaben = pd.read_csv(folder_preprocessed_data /
                          'df_ausgaben_all_merged.csv',
                          index_col=0)

In [11]:
df_filtered = df_ausgaben[df_ausgaben.year == 2000]

In [12]:
df_filtered[df_filtered['canton'] == 'ag']['0'].values[0]

251255.47496

In [13]:
'aa'.upper()

'AA'